In [0]:
%sql
SELECT column_name, data_type
FROM nyc_taxi.information_schema.columns
WHERE table_name = 'bronze_data'
  AND table_schema = 'default';

column_name,data_type
VendorID,INT
tpep_pickup_datetime,TIMESTAMP
tpep_dropoff_datetime,TIMESTAMP
passenger_count,LONG
trip_distance,DOUBLE
RateCodeID,LONG
store_and_fwd_flag,STRING
PULocationID,INT
DOLocationID,INT
payment_type,LONG


### 1. Find the total number of trips in Jan 2015 and compare with Jan–Mar 2016.

In [0]:
%sql
select count(*) as number_of_trips,year(tpep_pickup_datetime) as year,month(tpep_pickup_datetime) as month from nyc_taxi.default.silver_data group by year,month order by year,month

number_of_trips,year,month
3475233,2025,1
3577542,2025,2
4145229,2025,3
3970568,2025,4
4591844,2025,5
4322949,2025,6
3898971,2025,7
3574074,2025,8
1,2025,9


### 2. Calculate the average trip distance and average fare amount.

In [0]:
%sql
select concat(round(avg(trip_distance),2)," miles") as average_trip_distance,concat("$",round(avg(fare_amount),2)) as average_fare_amount from nyc_taxi.default.silver_data;

average_trip_distance,average_fare_amount
6.89 miles,$17.99


### 3.Find the busiest pickup hour of the day.

In [0]:
%sql
select hour(tpep_pickup_datetime) as Busiest_hour,count(*) as number_of_trips from nyc_taxi.default.silver_data group by Busiest_hour order by number_of_trips desc limit 1

Busiest_hour,number_of_trips
18,2277974


### 4. Calculate the total revenue per month

In [0]:
%sql
select year(tpep_pickup_datetime) as year,month(tpep_pickup_datetime) as month,sum(total_amount) as total_revenue from nyc_taxi.default.silver_data where total_amount > 0 group by year,month order by year,month

year,month,total_revenue
2025,1,9.066274142002532E7
2025,2,9.10447134600153E7
2025,3,1.1081291951003332E8
2025,4,1.0770823574001488E8
2025,5,1.2594822167000735E8
2025,6,1.206329495000672E8
2025,7,1.0696111712005189E8
2025,8,9.677493851003136E7
2025,9,25.62


### 5. Find the top 5 days with highest total revenue.

In [0]:
%sql
select date(tpep_pickup_datetime) as date,round(sum(total_amount),1) as total_revenue from nyc_taxi.default.silver_data where total_amount>0 group by date order by total_revenue desc limit 5

date,total_revenue
2025-06-05,4983721.3
2025-05-15,4906815.0
2025-05-17,4887331.9
2025-05-16,4810421.1
2025-05-14,4750108.4


### 6. Top 10 most common pick ups

In [0]:
%sql
select PULocationID,count(*) as total_trips from nyc_taxi.default.silver_data group by PULocationID order by total_trips desc limit 10

PULocationID,total_trips
161,1383279
132,1351013
237,1340920
236,1184847
230,1013928
186,1004135
162,982329
142,884218
170,850805
234,843038


### 7. Top 10 most common drops

In [0]:
%sql
select DOLocationID,count(*) as total_trips from nyc_taxi.default.silver_data group by DOLocationID order by total_trips desc limit 10

DOLocationID,total_trips
236,1236909
237,1218091
161,1130272
230,968268
170,884650
162,840513
68,811282
142,788758
239,785878
234,748807


### 8. Find the longest trip (by distance) and highest fare trip.

In [0]:
%sql
-- Longest Trip Details
SELECT *
FROM nyc_taxi.default.silver_data
WHERE trip_distance = (
    SELECT MAX(trip_distance) FROM nyc_taxi.default.silver_data
);

-- Highest Fare Trip Details
SELECT *
FROM nyc_taxi.default.silver_data
WHERE fare_amount = (
    SELECT MAX(fare_amount) FROM nyc_taxi.default.silver_data
);


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RateCodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
1,2025-01-20T12:07:18.000Z,2025-01-20T12:12:42.000Z,1,1.6,1,N,138,8,4,863372.12,6.75,0.5,0.0,0.0,1.0,863380.37,0.0,1.75,0.0


### 9. Calculate the percentage of trips with passenger_count > 3.

In [0]:
%sql
select
round(
(count(case when passenger_count>3 then 1 end) * 100.0) / count(*) ,2) as percent
from nyc_taxi.default.silver_data

percent
2.37


### 10. Identify if extra charges (MTA_tax, surcharge) significantly affect final fare.

In [0]:
%sql
select case when (extra+mta_tax+improvement_surcharge)>0 then "Extra" else "No extra" end as affecting,count(*) as total_trips,sum(total_amount) as amount from nyc_taxi.default.silver_data where total_amount>0 group by affecting;

affecting,total_trips,amount
No extra,4450,139828.40000000002
Extra,30936787,8.504060341502845E8
